In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import bruges as bg
import pyvista as pv

In [ ]:
filename_valdemar =   "C:/Users/vwt756/OneDrive - GEUS/MLO documents/pythoncode_local/LOCRETA/data/seismic/Valdemar_full_stack_scaled_1.sgy"
filename_faultpred = 'C:/Users/vwt756/OneDrive - GEUS/MLO documents/pythoncode_local/LOCRETA/data/seismic/Valdemar_full_faultpred.sgy'

filename_orig = '//netapp/Petrel/PetrelProjects/UTM33/Employee/PSpro/DATA/Mads/DATA/seismic/from_kenb/Valdemar_full_stack.sgy'
path_attributes =  "M:/LOCRETA_WORKPACKAGE_D/Valdemar_sprint_well_data_Mads_L/MLO documents/pythoncode_local/LOCRETA/data/attributes/"

from segysak.segy import segy_loader
valdemar = segy_loader(filename_valdemar ,cdp=None,  iline=189, xline=193,cdpx=181, cdpy = 185, offset=None, vert_domain='TWT', data_type='AMP')
valdemar_fault = segy_loader(filename_faultpred ,cdp=None,  iline=189, xline=193,cdpx=181, cdpy = 185, offset=None, vert_domain='TWT', data_type='AMP')
#valdemar_orig =  segy_loader(filename_orig ,cdp=None,  iline=189, xline=193,cdpx=181, cdpy = 185, offset=None, vert_domain='TWT', data_type='AMP')
#chaos = segy_loader(path_attributes + 'Variance_Cube.sgy' ,cdp=None,  iline=189, xline=193,cdpx=181, cdpy = 185, offset=None, vert_domain='TWT', data_type='AMP')
#edge =  segy_loader(path_attributes + '3D Edge.sgy' ,cdp=None,  iline=189, xline=193,cdpx=181, cdpy = 185, offset=None, vert_domain='TWT', data_type='AMP')
#ant =   segy_loader(path_attributes + 'Variance_cube_ant-tracked_passive.sgy' ,cdp=None,  iline=189, xline=193,cdpx=181, cdpy = 185, offset=None, vert_domain='TWT', data_type='AMP')


In [ ]:
valdemar["CNN"] = (("iline", "xline", "twt"), valdemar_fault.data.data)

In [ ]:
from scipy.interpolate import griddata
mask_below = valdemar.where(valdemar.twt < tuxen_horizon_mapped.twt + 5)
mask_above = valdemar.where(valdemar.twt > tuxen_horizon_mapped.twt - 5)
masks = [mask_above, mask_below]

masked_data = 0.5 * (mask_below + mask_above)

summed_amp = masked_data.sum(dim="twt")


xi = np.linspace(tuxen_horizon.cdp_x.min(), tuxen_horizon.cdp_x.max(), 2500)
yi = np.linspace(tuxen_horizon.cdp_y.min(), tuxen_horizon.cdp_y.max(), 2500)
X, Y = np.meshgrid(xi, yi)
Z = griddata((tuxen_horizon.cdp_x, tuxen_horizon.cdp_y), tuxen_horizon.twt, (X, Y))

In [ ]:
path_faults =  "C:/Users/vwt756/OneDrive - GEUS/MLO documents/pythoncode_local/LOCRETA/data/faults/"

faultsticks = pd.DataFrame(columns = [ 'utm_x','utm_y', 'twt', 'fault_id'] )

from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(path_faults) if isfile(join(path_faults, f))]


#pd.read_csv(path_horizons+hrz_file,names=["cdp_x","cdp_y","twt"], sep='\s+')
temp=[]

for i in onlyfiles:
    temp_file = pd.read_csv(path_faults + i, sep='\s+',usecols=[3,4,5], names=[ 'utm_x','utm_y', 'twt'])
    temp_file['fault_id'] = i
    #print(i)
    faultsticks = faultsticks.append(temp_file)
    
faultsticks['fault_id'] = faultsticks['fault_id'].astype('float64')

# UTM coords of 3 unique inline, crossline locations.
corners_xy = np.array([[595324.50, 6.178583*10**6],
                       [599699.50, 6.178583*10**6],
                       [595324.50, 6.186083*10**6],
                       [599699.50, 6.186083*10**6]])

# The inline, crossline locations you just provided.
corners_ix = np.array([[7900, 7800],
                       [7900, 8150],
                       [8500, 7800],
                       [8500, 8150]])

transform = bg.transform.CoordTransform(corners_ix, corners_xy)

#C 7 1: INLINE 7900, CROSSLINE 7800, UTM-X 595324.50 M, UTM-Y 6.178583E+6 M
#C 8 2: INLINE 7900, CROSSLINE 8150, UTM-X 599699.50 M, UTM-Y 6.178583E+6 M
#C 9 3: INLINE 8500, CROSSLINE 7800, UTM-X 595324.50 M, UTM-Y 6.186083E+6 M
#C10 4: INLINE 8500, CROSSLINE 8150, UTM-X 599699.50 M, UTM-Y 6.186083E+6 M

faultsticks['iline'] = 0
faultsticks['xline'] = 0

xl = np.zeros([faultsticks.shape[0]])
il = np.zeros([faultsticks.shape[0]])

for i in range(faultsticks.shape[0]):
    a,b = transform.reverse([faultsticks['utm_x'].values[i], faultsticks['utm_y'].values[i]])
    il[i] = a
    xl[i] = b
    
faultsticks['iline'] = il
faultsticks['xline'] = xl

In [ ]:
d_orig = {}
for x in faultsticks['fault_id'].unique():
    d_orig["fault_{0}".format(int(x))] = pv.PolyData(faultsticks.loc[faultsticks['fault_id'] == x][['iline','xline','twt']].values).delaunay_3d()

In [ ]:
faultsticks = faultsticks.loc[faultsticks['twt']<2650]
faultsticks = faultsticks.loc[(faultsticks['iline']<valdemar.iline.data.max()) & (faultsticks['iline']>valdemar.iline.data.min())]
faultsticks = faultsticks.loc[(faultsticks['xline']<valdemar.xline.data.max()) & (faultsticks['xline']>valdemar.xline.data.min())]

In [ ]:
IL = 8100

In [ ]:
d = {}
for x in faultsticks['fault_id'].unique():
    d["fault_{0}".format(int(x))] = pv.PolyData(faultsticks.loc[faultsticks['fault_id'] == x][['iline','xline','twt']].values).delaunay_3d()

## horizons

In [ ]:
path_horizons =  "C:/Users/vwt756/OneDrive - GEUS/MLO documents/pythoncode_local/LOCRETA/data/horizons/"

from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(path_horizons) if isfile(join(path_horizons, f))]

hrz_list = {}
hrz_3d = {}

k=0
for i in onlyfiles:
    hrz_file = i
    extension = '.txt'
    if extension in hrz_file:
        hrz = hrz_file.replace(extension,'')
    hrz_list[hrz] = pd.read_csv(path_horizons+hrz_file,names=["cdp_x","cdp_y","twt"], sep='\s+',skiprows = 10)
    

    xl = np.zeros([hrz_list[hrz].shape[0]])
    il = np.zeros([hrz_list[hrz].shape[0]])

    for i in range(hrz_list[hrz].shape[0]):
        a,b = transform.reverse([hrz_list[hrz]['cdp_x'].values[i], hrz_list[hrz]['cdp_y'].values[i]])
        il[i] = a
        xl[i] = b

    hrz_list[hrz]['iline'] = il
    hrz_list[hrz]['xline'] = xl
    
    hrz_3d[hrz] = pv.PolyData(hrz_list[hrz][['iline','xline','twt']].values).delaunay_2d()
    hrz_3d[hrz] = hrz_3d[hrz].smooth(n_iter=2000)
    print(hrz)
    k=k+1


In [ ]:
t_start = 1900 // 4 - 1500 //4
t_stop = 2650 // 4  - 1500 //4

In [ ]:
seis = valdemar.data.data / np.max(np.abs(valdemar.data.data))
fault = valdemar.CNN.where(valdemar.CNN>0.9).data

seis = seis[:,:,t_start:t_stop]
fault = fault[:,:,t_start:t_stop]

I,X,T=seis.shape
dims=(I,X,T)
spacing=(2,1,4)
origin=(7900,7800,1900)
mesh = pv.UniformGrid(dims,spacing,origin)
mesh2 = pv.UniformGrid(dims,spacing,origin)

#mesh.scale([1000.0, 1000.0, 1000.0])
#mesh2.scale([1000.0, 1000.0, 1000.0])

mesh['seis']=seis.ravel('F')
mesh2['CNN Fault probability'] = fault.ravel('F')

In [ ]:
color_hrz = ['#1f77b4', 'm', 'r', 'cyan', 'g', 'orange']
label_hrz = ['BCU' , 'Base Chalk', 'Top Sola', 'Top Chalk', 'Top Tuxen', 'Top Valhall']

# Plotting

## Inline section

In [ ]:
slc = mesh.slice_orthogonal(x=0,  y=7975, z=2600)
slc2 = mesh2.slice_orthogonal(x=0,  y=7975, z=2601)


p = pv.Plotter(notebook=False)
#p.set_scale(xscale=2, yscale = 2)
#p.add_mesh(mesh, opacity=0.2, cmap='Greys')
p.add_mesh(slc, cmap='Greys', opacity = 1, lighting=False, clim=[-1,1], show_scalar_bar =False)
p.add_mesh(slc2, nan_color = None, nan_opacity= 0 , lighting=False, cmap='Reds' , label='CNN Fault',
           scalar_bar_args = dict(    title_font_size=20,
    label_font_size=16, n_labels=3,  fmt="%.2f", interactive=True), show_scalar_bar =False)#, clim=[0.9,1], opacity=0.7)

#for i in faultsticks.loc[faultsticks['iline'] == IL]['fault_id'].unique():   
#    p.add_mesh(d['fault_'+str(int(i))])

#for i in faultsticks['fault_id'].unique():   
#    p.add_mesh(d['fault_'+str(int(i))].slice_orthogonal(x=0, y=7975, z=2600), color='lawngreen',line_width=2, show_edges=True, edge_color='k', lighting=False)

p.show_bounds(slc, xlabel='Inline #', ylabel='Crossline #', zlabel='TWT', 
              grid='front', location='outer', bounds = [7900, 8500,
    7800, 8150, 2650, 1900], color='k', fmt="%.1f")

#k = 0
#for hrz in hrz_list:
#    p.add_mesh(hrz_3d[hrz].slice_orthogonal(x=0, y=7975, z=2600), color = color_hrz[k] ,line_width=4, show_edges=True, edge_color='k', lighting=False, nan_color = None, nan_opacity= 0)
#    k += 1
p.camera_position = 'xz'
#p.camera_position = [(7795.5509416442255, 8662.157536121902, 1600.897820095156),
# (8179.006405051869, 7946.098887495095, 2442.14694664309),
# (0.16163001548980183, -0.19743229153787573, -0.9669003197594206)]
p.set_background('white')
p.show()

In [ ]:
p.camera_position
#[(7795.5509416442255, 8662.157536121902, 1600.897820095156),
# (8179.006405051869, 7946.098887495095, 2442.14694664309),
# (0.16163001548980183, -0.19743229153787573, -0.9669003197594206)]

In [ ]:
slc = mesh.slice_orthogonal(x=0,  y=7975, z=2601)
slc2 = mesh2.slice_orthogonal(x=0,  y=7975, z=2600)


p = pv.Plotter(notebook=False)
p.set_scale(xscale=4, yscale = 4)
#p.add_mesh(mesh, opacity=0.2, cmap='Greys')
p.add_mesh(slc, cmap='Greys', opacity = 1, lighting=False, clim=[-1,1], show_scalar_bar =False)
#p.add_mesh(slc2, nan_color = None, nan_opacity= 0 , lighting=False, cmap='Reds' , label='CNN Fault',
#           scalar_bar_args = dict(    title_font_size=20,
#    label_font_size=16, n_labels=3,  fmt="%.2f", interactive=True), show_scalar_bar =False)#, clim=[0.9,1], opacity=0.7)

#for i in faultsticks.loc[faultsticks['iline'] == IL]['fault_id'].unique():   
#    p.add_mesh(d['fault_'+str(int(i))])

#for i in faultsticks['fault_id'].unique():   
#    p.add_mesh(d['fault_'+str(int(i))].slice_orthogonal(x=0, y=7975, z=2600), color='lawngreen',line_width=2, show_edges=True, edge_color='k', lighting=False)

p.show_bounds(slc, xlabel='Inline #', ylabel='Crossline #', zlabel='TWT', 
              grid='front', location='outer', bounds = [7900, 8500,
    7800, 8150, 2650, 1900], color='k', fmt="%.1f")

k = 0
for hrz in hrz_list:
    p.add_mesh(hrz_3d[hrz].slice_orthogonal(x=0, y=7975, z=2600), color = color_hrz[k] ,line_width=2, show_edges=True, edge_color='k', lighting=False, nan_color = None, nan_opacity= 0)
    k += 1
p.camera_position = 'xz'
#p.camera_position = [(7795.5509416442255, 8662.157536121902, 1600.897820095156),
# (8179.006405051869, 7946.098887495095, 2442.14694664309),
# (0.16163001548980183, -0.19743229153787573, -0.9669003197594206)]
p.set_background('white')
p.show()

## Xline section

In [ ]:
IL = 8440

slc = mesh.slice_orthogonal(x=IL,  y=0, z=2601)
slc2 = mesh2.slice_orthogonal(x=IL,  y=0, z=2600)

p = pv.Plotter(notebook=False, polygon_smoothing=True)
p.set_scale(xscale=4, yscale = 4)
#p.add_mesh(mesh, opacity=0.2, cmap='Greys')
p.add_mesh(slc, cmap='Greys', opacity = 1, lighting=False, clim=[-1,1], show_scalar_bar =False)
p.add_mesh(slc2, nan_color = None, nan_opacity= 0 , lighting=False, cmap='Reds' , label='CNN Fault',
           scalar_bar_args = dict(    title_font_size=20,
    label_font_size=16, n_labels=3,  fmt="%.2f", interactive=True), show_scalar_bar =False)#, clim=[0.9,1], opacity=0.7)

#for i in faultsticks.loc[faultsticks['iline'] == IL]['fault_id'].unique():   
#    p.add_mesh(d['fault_'+str(int(i))])

for i in faultsticks['fault_id'].unique():   
    p.add_mesh(d['fault_'+str(int(i))].slice_orthogonal(x=IL, y=0, z=2600), color='lawngreen',line_width=2, show_edges=True, edge_color='k', lighting=False)

p.show_bounds(slc, xlabel='Inline #', ylabel='Crossline #', zlabel='TWT', 
              grid='front', location='outer', bounds = [7900, 8500,
    7800, 8150, 2650, 1900], color='k', fmt="%.1f")


 #mesh.slice(normal=[1, 1, 0])
    
    
k = 0
for hrz in hrz_list:
    p.add_mesh(hrz_3d[hrz].slice_orthogonal(x=IL, y=0, z=2600) ,color = color_hrz[k] ,line_width=2, show_edges=True, edge_color='k', lighting=False, nan_color = None, nan_opacity= 0)
    k += 1
#for hrz in hrz_list:
#    p.add_mesh(hrz_3d[hrz].slice_orthogonal(x=8440, y=0, z=2600) ,color = color_hrz[k] ,line_width=2, show_edges=True, edge_color='k', lighting=False, nan_color = None, nan_opacity= 0)
#    k += 1
    
#p.camera_position = [(7486.767661866813, 8287.661754710181, 1803.2750145811142),
# (8170.985651254801, 7942.297348728063, 2469.038549863768),
# (0.19330745322599918, -0.08796346563670202, -0.9771871147535969)]
#p.enable_parallel_projection()

#p.add_actor(actor)
#p.set_background("royalblue", top="aliceblue")
p.set_background('white')
p.show()

In [ ]:
p.camera_position
#[(7486.767661866813, 8287.661754710181, 1803.2750145811142),
# (8170.985651254801, 7942.297348728063, 2469.038549863768),
# (0.19330745322599918, -0.08796346563670202, -0.9771871147535969)]

In [ ]:


slc = mesh.slice_orthogonal(x=IL,  y=0, z=2601)
slc2 = mesh2.slice_orthogonal(x=IL,  y=0, z=2600)

p = pv.Plotter(notebook=False, polygon_smoothing=True)
p.set_scale(xscale=4, yscale = 4)
#p.add_mesh(mesh, opacity=0.2, cmap='Greys')
p.add_mesh(slc, cmap='Greys', opacity = 1, lighting=False, clim=[-1,1], show_scalar_bar =False)
#p.add_mesh(slc2, nan_color = None, nan_opacity= 0 , lighting=False, cmap='Reds' , label='CNN Fault',
#           scalar_bar_args = dict(    title_font_size=20,
#    label_font_size=16, n_labels=3,  fmt="%.2f", interactive=True), show_scalar_bar =False)#, clim=[0.9,1], opacity=0.7)

#for i in faultsticks.loc[faultsticks['iline'] == IL]['fault_id'].unique():   
#    p.add_mesh(d['fault_'+str(int(i))])

#for i in faultsticks['fault_id'].unique():   
#    p.add_mesh(d['fault_'+str(int(i))].slice_orthogonal(x=IL, y=0, z=2600), color='lawngreen',line_width=2, show_edges=True, edge_color='k', lighting=False)

p.show_bounds(slc, xlabel='Inline #', ylabel='Crossline #', zlabel='TWT', 
              grid='front', location='outer', bounds = [7900, 8500,
    7800, 8150, 2650, 1900], color='k', fmt="%.1f")


 #mesh.slice(normal=[1, 1, 0])
    
    
k = 0
for hrz in hrz_list:
    p.add_mesh(hrz_3d[hrz].slice_orthogonal(x=IL, y=0, z=2600) ,color = color_hrz[k] ,line_width=2, show_edges=True, edge_color='k', lighting=False, nan_color = None, nan_opacity= 0)
    k += 1
#for hrz in hrz_list:
#    p.add_mesh(hrz_3d[hrz].slice_orthogonal(x=8440, y=0, z=2600) ,color = color_hrz[k] ,line_width=2, show_edges=True, edge_color='k', lighting=False, nan_color = None, nan_opacity= 0)
#    k += 1
    
p.camera_position = [(7486.767661866813, 8287.661754710181, 1803.2750145811142),
 (8170.985651254801, 7942.297348728063, 2469.038549863768),
 (0.19330745322599918, -0.08796346563670202, -0.9771871147535969)]
#p.enable_parallel_projection()

#p.add_actor(actor)
#p.set_background("royalblue", top="aliceblue")
p.set_background('white')
p.show()

In [ ]:
p.camera_position
#[(7486.767661866813, 8287.661754710181, 1803.2750145811142),
# (8170.985651254801, 7942.297348728063, 2469.038549863768),
# (0.19330745322599918, -0.08796346563670202, -0.9771871147535969)]

## Interactive Sessions

In [ ]:
p = pv.Plotter(notebook=False)
#p.set_scale(yscale=3)
#p.add_mesh(mesh, opacity=0.2, cmap='Greys')
p.add_mesh_slice(mesh, cmap='Greys', opacity = 1, lighting=False, clim=[-1,1], show_scalar_bar =False)
#p.add_mesh_slice(mesh2, nan_color = None, nan_opacity= 0 , lighting=False, cmap='Reds' , label='CNN Fault',
#           scalar_bar_args = dict(    title_font_size=20,
#    label_font_size=16, n_labels=3,  fmt="%.2f", interactive=True), show_scalar_bar =False)#, clim=[0.9,1], opacity=0.7)

#for i in faultsticks.loc[faultsticks['iline'] == IL]['fault_id'].unique():   
#    p.add_mesh(d['fault_'+str(int(i))])

#for i in faultsticks['fault_id'].unique():   
#    p.add_mesh(d['fault_'+str(int(i))].slice_orthogonal(x=8440, y=0, z=2600), color='lawngreen',line_width=2, show_edges=True, edge_color='k', lighting=False)
#
#p.show_bounds(slc, xlabel='Inline #', ylabel='Crossline #', zlabel='TWT', 
#              grid='front', location='outer', bounds = [7900, 8500,
#    7800, 8150, 2650, 1900], color='k', fmt="%.1f")



#k = 0
#for hrz in hrz_list:
#    p.add_mesh(hrz_3d[hrz], color = color_hrz[k] ,line_width=2, show_edges=True, edge_color='k', lighting=False, nan_color = None, nan_opacity= 0)
#    k += 1
#p.camera_position = 'xz'
#p.enable_parallel_projection()

#p.add_actor(actor)
#p.set_background("royalblue", top="aliceblue")
p.set_background('grey')
p.show()

In [ ]:
p = pv.Plotter(notebook=False, polygon_smoothing=True)
p.add_mesh_slice_orthogonal(mesh,cmap='Greys', opacity = 1, lighting=False, clim=[-1,1], show_scalar_bar =False)

for i in faultsticks['fault_id'].unique():  
    p.add_mesh(d_orig['fault_'+str(int(i))], color='lawngreen', opacity = 0.5)
p.set_scale(yscale=3)
p.show_bounds(slc, xlabel='Inline #', ylabel='Crossline #', zlabel='TWT', 
              grid='front', location='outer', bounds = [7900, 8500,
    7800, 8150, 2650, 1900], color='white', fmt="%.1f")
p.show()

In [ ]:
d_orig['fault_1']

In [ ]:
# Interpolation test
p = pv.Plotter(notebook=False, polygon_smoothing=True)
p.add_mesh_slice_orthogonal(mesh,cmap='Greys', opacity = 1, lighting=False, clim=[-1,1], show_scalar_bar =False)
plane = d_orig['fault_1'].interpolate(mesh)
p.add_mesh(plane, style='wireframe', line_width=5)
p.set_scale(yscale=3)
p.show_bounds(slc, xlabel='Inline #', ylabel='Crossline #', zlabel='TWT', 
              grid='front', location='outer', bounds = [7900, 8500,
    7800, 8150, 2650, 1900], color='white', fmt="%.1f")
p.show()

In [ ]:
plane

In [ ]:
p = pv.Plotter(notebook=False)

def create_mesh(iline):
    iline_ = int(iline)
    slc = mesh.slice_orthogonal(x=iline_,  y=0, z=2601)
    p.add_mesh(slc, cmap='Greys', opacity = 1, lighting=False, clim=[-1,1], show_scalar_bar =False)
    return

p.add_slider_widget(create_mesh, [7900, 8500], title='Inline')
p.set_scale(yscale=3)
p.show()


In [ ]:
p = pv.Plotter(notebook=False)
p.set_scale(xscale=4, yscale = 4)
p.add_mesh(hrz_3d['BaseChalk_horizon'])
p.show_bounds(slc, xlabel='Inline #', ylabel='Crossline #', zlabel='TWT', 
              grid='front', location='outer', bounds = [7900, 8500,
    7800, 8150, 2650, 1900], color='white', fmt="%.1f")
p.camera_position = [(7486.767661866813, 8287.661754710181, 1803.2750145811142),
 (8170.985651254801, 7942.297348728063, 2469.038549863768),
 (0.19330745322599918, -0.08796346563670202, -0.9771871147535969)]
p.show()

# 2D sections

IL

In [ ]:
IL = 8040

slc = mesh.slice(origin=(IL,  0, 2601))
slc2 = mesh2.slice(origin=(IL,  0, 2600))

p = pv.Plotter(notebook=False, polygon_smoothing=True)
p.set_scale(xscale=4, yscale = 4)
#p.add_mesh(mesh, opacity=0.2, cmap='Greys')
p.add_mesh(slc, cmap='Greys', opacity = 1, lighting=False, clim=[-1,1], show_scalar_bar =False)
p.add_mesh(slc2, nan_color = None, nan_opacity= 0 , lighting=False, cmap='Reds' , label='CNN Fault',
           scalar_bar_args = dict(    title_font_size=20,
    label_font_size=16, n_labels=3,  fmt="%.2f", interactive=True), show_scalar_bar =False)#, clim=[0.9,1], opacity=0.7)

#for i in faultsticks.loc[faultsticks['iline'] == IL]['fault_id'].unique():   
#    p.add_mesh(d['fault_'+str(int(i))])

for i in faultsticks['fault_id'].unique():   
    p.add_mesh(d['fault_'+str(int(i))].slice_orthogonal(x=IL, y=0, z=0), color='lawngreen',line_width=2, show_edges=True, edge_color='k', lighting=False)

p.show_bounds(slc, xlabel='Inline #', ylabel='Crossline #', zlabel='TWT', 
              location='outer', bounds = [7900, 8500,
    7800, 8150, 2650, 1900], color='k', fmt="%.1f")
#
#
# #mesh.slice(normal=[1, 1, 0])
#    
#    
k = 0
for hrz in hrz_list:
    p.add_mesh(hrz_3d[hrz].slice(origin=(IL,  0, 2600)) ,color = color_hrz[k] ,line_width=2, show_edges=True, edge_color='k', lighting=False, nan_color = None, nan_opacity= 0)
    k += 1
##for hrz in hrz_list:
##    p.add_mesh(hrz_3d[hrz].slice_orthogonal(x=8440, y=0, z=2600) ,color = color_hrz[k] ,line_width=2, show_edges=True, edge_color='k', lighting=False, nan_color = None, nan_opacity= 0)
##    k += 1
#    
#p.camera_position = [(7486.767661866813, 8287.661754710181, 1803.2750145811142),
# (8170.985651254801, 7942.297348728063, 2469.038549863768),
# (0.19330745322599918, -0.08796346563670202, -0.9771871147535969)]
#p.enable_parallel_projection()
#p.camera_position = [(8065.1693872481, 7975.0, 2272.0),
# (8440.0, 7975.0, 2272.0),
# (0.0, 0.0, -1.0)]
#p.add_actor(actor)
#p.set_background("royalblue", top="aliceblue")
p.set_background('white')
p.show()

In [ ]:

slc = mesh.slice(origin=(IL,  0, 2601))
slc2 = mesh2.slice(origin=(IL,  0, 2600))

p = pv.Plotter(notebook=False, polygon_smoothing=True)
p.set_scale(xscale=4, yscale = 4)
#p.add_mesh(mesh, opacity=0.2, cmap='Greys')
p.add_mesh(slc, cmap='Greys', opacity = 1, lighting=False, clim=[-1,1], show_scalar_bar =False)
#p.add_mesh(slc2, nan_color = None, nan_opacity= 0 , lighting=False, cmap='Reds' , label='CNN Fault',
#           scalar_bar_args = dict(    title_font_size=20,
#    label_font_size=16, n_labels=3,  fmt="%.2f", interactive=True), show_scalar_bar =False)#, clim=[0.9,1], opacity=0.7)

#for i in faultsticks.loc[faultsticks['iline'] == IL]['fault_id'].unique():   
#    p.add_mesh(d['fault_'+str(int(i))])

#for i in faultsticks['fault_id'].unique():   
#    p.add_mesh(d['fault_'+str(int(i))].slice_orthogonal(x=IL, y=0, z=0), color='lawngreen',line_width=2, show_edges=True, edge_color='k', lighting=False)
#
p.show_bounds(slc, xlabel='Inline #', ylabel='Crossline #', zlabel='TWT', 
              location='outer', bounds = [7900, 8500,
    7800, 8150, 2650, 1900], color='k', fmt="%.1f")
#
#
# #mesh.slice(normal=[1, 1, 0])
#    
#    
k = 0
for hrz in hrz_list:
    p.add_mesh(hrz_3d[hrz].slice(origin=(IL,  0, 2600)) ,color = color_hrz[k] ,line_width=2, show_edges=True, edge_color='k', lighting=False, nan_color = None, nan_opacity= 0)
    k += 1
##for hrz in hrz_list:
##    p.add_mesh(hrz_3d[hrz].slice_orthogonal(x=8440, y=0, z=2600) ,color = color_hrz[k] ,line_width=2, show_edges=True, edge_color='k', lighting=False, nan_color = None, nan_opacity= 0)
##    k += 1
#    
#p.camera_position = [(7486.767661866813, 8287.661754710181, 1803.2750145811142),
# (8170.985651254801, 7942.297348728063, 2469.038549863768),
# (0.19330745322599918, -0.08796346563670202, -0.9771871147535969)]
#p.enable_parallel_projection()
p.camera_position = [(8065.1693872481, 7975.0, 2272.0),
 (8440.0, 7975.0, 2272.0),
 (0.0, 0.0, -1.0)]
#p.add_actor(actor)
#p.set_background("royalblue", top="aliceblue")
p.set_background('white')
p.show()

XL

In [ ]:
XL = 7975

slc = mesh.slice(normal='y', origin =(0,  XL, 2600))
slc2 = mesh2.slice(normal='y', origin =(0,  XL, 2601))


p = pv.Plotter(notebook=False)
p.set_scale(xscale=4, yscale = 4)
#p.add_mesh(mesh, opacity=0.2, cmap='Greys')
p.add_mesh(slc, cmap='Greys', opacity = 1, lighting=False, clim=[-1,1], show_scalar_bar =False)
p.add_mesh(slc2, nan_color = None, nan_opacity= 0 , lighting=False, cmap='Reds' , label='CNN Fault',
           scalar_bar_args = dict(    title_font_size=20,
    label_font_size=16, n_labels=3,  fmt="%.2f", interactive=True), show_scalar_bar =False)#, clim=[0.9,1], opacity=0.7)

#for i in faultsticks.loc[faultsticks['iline'] == IL]['fault_id'].unique():   
#    p.add_mesh(d['fault_'+str(int(i))])

for i in faultsticks['fault_id'].unique():   
    p.add_mesh(d['fault_'+str(int(i))].slice_orthogonal(0,  XL, 0), color='lawngreen',line_width=2, show_edges=True, edge_color='k', lighting=False)

p.show_bounds(slc, xlabel='Inline #', ylabel='Crossline #', zlabel='TWT', 
              location='outer', bounds = [7900, 8500,
    7800, 8150, 2650, 1900], color='k', fmt="%.1f")

k = 0
for hrz in hrz_list:
    p.add_mesh(hrz_3d[hrz].slice(normal='y',origin=(0, XL, 2600)), color = color_hrz[k] ,line_width=2, show_edges=True, edge_color='k', lighting=False, nan_color = None, nan_opacity= 0)
    k += 1
p.camera_position = 'zx'
#p.camera_position = [(7795.5509416442255, 8662.157536121902, 1600.897820095156),
# (8179.006405051869, 7946.098887495095, 2442.14694664309),
# (0.16163001548980183, -0.19743229153787573, -0.9669003197594206)]
p.set_background('white')
p.show()

In [ ]:
p.camera_position
#[(7486.767661866813, 8287.661754710181, 1803.2750145811142),
# (8170.985651254801, 7942.297348728063, 2469.038549863768),
# (0.19330745322599918, -0.08796346563670202, -0.9771871147535969)]

In [ ]:
XL = 7975

slc = mesh.slice(normal='y', origin =(0,  XL, 2600))
slc2 = mesh2.slice(normal='y', origin =(0,  XL, 2601))


p = pv.Plotter(notebook=False)
p.set_scale(xscale=4, yscale = 4)
#p.add_mesh(mesh, opacity=0.2, cmap='Greys')
p.add_mesh(slc, cmap='Greys', opacity = 1, lighting=False, clim=[-1,1], show_scalar_bar =False)
#p.add_mesh(slc2, nan_color = None, nan_opacity= 0 , lighting=False, cmap='Reds' , label='CNN Fault',
#           scalar_bar_args = dict(    title_font_size=20,
#    label_font_size=16, n_labels=3,  fmt="%.2f", interactive=True), show_scalar_bar =False)#, clim=[0.9,1], opacity=0.7)

#for i in faultsticks.loc[faultsticks['iline'] == IL]['fault_id'].unique():   
#    p.add_mesh(d['fault_'+str(int(i))])

#for i in faultsticks['fault_id'].unique():   
#    p.add_mesh(d['fault_'+str(int(i))].slice_orthogonal(0,  XL, 0), color='lawngreen',line_width=2, show_edges=True, edge_color='k', lighting=False)

p.show_bounds(slc, xlabel='Inline #', ylabel='Crossline #', zlabel='TWT', 
              location='outer', bounds = [7900, 8500,
    7800, 8150, 2650, 1900], color='k', fmt="%.1f")

k = 0
for hrz in hrz_list:
    p.add_mesh(hrz_3d[hrz].slice(normal='y',origin=(0, XL, 2600)), color = color_hrz[k] ,line_width=2, show_edges=True, edge_color='k', lighting=False, nan_color = None, nan_opacity= 0)
    k += 1
p.camera_position = [(8200.0, 8594.207942319958, 2272.9695870149253),
 (8200.0, 7975.0, 2272.9695870149253),
 (0.002308863969307561, 0.0, -0.9999973345700334)]
#p.camera_position = [(7795.5509416442255, 8662.157536121902, 1600.897820095156),
# (8179.006405051869, 7946.098887495095, 2442.14694664309),
# (0.16163001548980183, -0.19743229153787573, -0.9669003197594206)]
p.set_background('white')
p.show()

In [ ]:
p = pv.Plotter(notebook=False)

def callback(point, i):
    mesh.points[i] = point

p.add_sphere_widget(callback, center=mesh.points)
p.add_mesh_clip_box(mesh, color='red')
p.show()

In [ ]:

pl = pv.Plotter(notebook=False)
def create_mesh(value):
    res = int(value)
    sphere = pv.Sphere(phi_resolution=res, theta_resolution=res)
    pl.add_mesh(sphere, name="sphere", show_edges=True)
slider = pl.add_slider_widget(
    create_mesh,
    [5, 100],
    title="Resolution",
    title_opacity=0.5,
    title_color="red",
    fmt="%0.9f",
    title_height=0.08,
)
pl.show()

In [ ]:
class MyCustomRoutine():
    def __init__(self, mesh):
        self.output = mesh # Expected PyVista mesh type
        # default parameters
        self.kwargs = {
            'radius': 0.5,
            'theta_resolution': 30,
            'phi_resolution': 30,
        }

    def __call__(self, param, value):
        self.kwargs[param] = value
        self.update()

    def update(self):
        # This is where you call your simulation
        result = pv.Sphere(**self.kwargs)
        self.output.overwrite(result)
        return

In [ ]:
starting_mesh = pv.Sphere()
engine = MyCustomRoutine(starting_mesh)

In [ ]:
p = pv.Plotter(notebook=False)
p.add_mesh(starting_mesh, show_edges=True)
p.add_slider_widget(
    callback=lambda value: engine('phi_resolution', int(value)),
    rng=[3, 60],
    value=30,
    title="Phi Resolution",
    pointa=(.025, .1), pointb=(.31, .1),
    style='modern',
)
p.add_slider_widget(
    callback=lambda value: engine('theta_resolution', int(value)),
    rng=[3, 60],
    value=30,
    title="Theta Resolution",
    pointa=(.35, .1), pointb=(.64, .1),
    style='modern',
)
p.add_slider_widget(
    callback=lambda value: engine('radius', value),
    rng=[0.1, 1.5],
    value=0.5,
    title="Radius",
    pointa=(.67, .1), pointb=(.98, .1),
    style='modern',
)
p.show()

In [ ]:
seis = valdemar.data.data / np.max(np.abs(valdemar.data.data))
fault = valdemar.CNN.where(valdemar.CNN>0.9).data

seis = seis[:,:,t_start:t_stop]
fault = fault[:,:,t_start:t_stop]

I,X,T=seis.shape
dims=(I,X,T)
spacing=(2,1,4)
origin=(7900,7800,1900)
mesh = pv.UniformGrid(dims,spacing,origin)
mesh2 = pv.UniformGrid(dims,spacing,origin)

mesh['seis']=seis.ravel('F')
mesh2['CNN Fault probability'] = fault.ravel('F')



slc = mesh.slice(normal='y', origin = (8100,7975,2599))
slc2 = mesh2.slice(normal='y', origin = (8100,7975,2599))

p = pv.Plotter()
#p.add_mesh(mesh, opacity=0.2, cmap='Greys')
p.add_mesh(slc, cmap='Greys', opacity = 1, lighting=False, clim=[-1,1], show_scalar_bar =False)
p.add_mesh(slc2, nan_color = None, nan_opacity= 0 , lighting=False, cmap='Reds' , label='CNN Fault')#, clim=[0.9,1], opacity=0.7)

#for i in faultsticks.loc[faultsticks['iline'] == IL]['fault_id'].unique():   
#    p.add_mesh(d['fault_'+str(int(i))])

for i in faultsticks['fault_id'].unique():   
    p.add_mesh(d['fault_'+str(int(i))].slice_orthogonal(x=0, y=7975, z=0), color='lawngreen',line_width=2, show_edges=True, edge_color='k', lighting=False)

    
#p.show_bounds( xlabel='Iline', ylabel='Xlne', zlabel='TWT', 
#              grid='front', location='outer', all_edges=True, minor_ticks=True)



k = 0
for hrz in hrz_list:
    p.add_mesh(hrz_3d[hrz].slice(normal='y', origin = (8100,7975,2599)), color = color_hrz[k] ,line_width=2, show_edges=True, edge_color='k', lighting=False, nan_color = None, nan_opacity= 0)
    k += 1
p.camera_position = 'xz'
#p.enable_parallel_projection()

#p.add_actor(actor)
p.set_background("royalblue", top="aliceblue")

p.show()

In [ ]:
cpos = [(540.9115516905358, -617.1912234499737, 180.5084853429126),
 (128.31920055083387, 126.4977720785509, 111.77682599082095),
 (-0.1065160140819035, 0.032750075477590124, 0.9937714884722322)]

p = pv.Plotter(shape=(2,2))
# XYZ - show 3D scene first
p.subplot(1,1)
p.add_mesh(slc)
p.show_grid()
p.camera_position = cpos
# XY
p.subplot(0,0)
p.add_mesh(slc)
p.show_grid()
p.camera_position = 'xy'
p.enable_parallel_projection()
# ZY
p.subplot(0,1)
p.add_mesh(slc)
p.show_grid()
p.camera_position = 'zy'
p.enable_parallel_projection()
# XZ
p.subplot(1,0)
p.add_mesh(slc)
p.show_grid()
p.camera_position = 'xz'
p.enable_parallel_projection()

p.show()

# Voxelize / volumetric

In [ ]:
cloud = pv.PolyData(fault)
cloud.plot(point_size=15)

In [ ]:
from pyvista import examples
mesh3 = examples.download_pine_roots()

In [ ]:
fault_test = np.nan_to_num(valdemar.CNN.where(valdemar.CNN>0.9).data)
mesh4 = pv.UniformGrid(dims,spacing,origin)
mesh4['fault'] = fault_test.ravel('F')
mesh4['fault']

In [ ]:
mesh3

In [ ]:
mesh2

In [ ]:
p = pv.Plotter()
p.add_mesh(largest, clim=[0.9,1])
#p.add_mesh(mesh4.outline())
p.show()

In [ ]:

p = pv.Plotter()
p.add_mesh(largest, nan_color = None, nan_opacity= 0  )
p.add_mesh(mesh4.outline())
3#p.camera_position = cpos
p.show()

In [ ]:
from pyvista import examples
surf = examples.load_airplane()
surf = surf.compute_cell_sizes(length=False, volume=False)
surf.plot(smooth_shading=True,notebook=False)

In [ ]:
surf

In [ ]:
#seis.shape

In [ ]:
#fault.shape

In [ ]:
prob_cube_90 = valdemar.CNN.where(valdemar.CNN>0.9).data
prob_cube_90 = np.nan_to_num(prob_cube_90)
bin_cube_90 = valdemar.CNN.where(valdemar.CNN>0.9).data
bin_cube_90 = np.nan_to_num(prob_cube_90)
bin_cube_90 = np.where(bin_cube_90<0.1,bin_cube_90,1)
bin_cube_90 = bin_cube_90.astype(np.uint8)

fault = prob_cube_90[:,:,t_start:t_stop]

I,X,T=seis.shape
dims=(I,X,T)
spacing=(2,1,4)
origin=(7900,7800,1900)
mesh = pv.UniformGrid(dims,spacing,origin)
mesh2 = pv.UniformGrid(dims,spacing,origin)
#
##mesh.scale([1000.0, 1000.0, 1000.0])
##mesh2.scale([1000.0, 1000.0, 1000.0])
#
#mesh['seis']=seis.ravel('F')
mesh2['CNN Fault probability'] = fault.ravel('F')

In [ ]:
opacity = [0, 0.0, 0, 0.75, 1.0]
mesh2.plot(volume=True, notebook=False,clim=[0,1], opacity= opacity)

In [ ]:
voxels = pv.voxelize(mesh2, density=100)

p = pv.Plotter()
p.add_mesh(voxels, color=True, show_edges=True, opacity=0.5)
#p.add_mesh(surface, color="lightblue", opacity=0.5)
p.show(cpos=cpos)

In [ ]:
largest = mesh2.connectivity(largest=False)
# or: largest = mesh.extract_largest()

p = pv.Plotter(notebook=False)
p.add_mesh(largest)
#p.add_mesh(mesh4.outline())
p.show()

In [ ]:
p = pv.Plotter(notebook=False)
p.add_mesh(largest, clim=[.5,1])
#p.add_mesh(mesh4.outline())
p.show()

In [ ]:
def generate_points(subset=0.02):
    """A helper to make a 3D NumPy array of points (n_points by 3)"""
    dataset = sumesh3rf
    ids = np.random.randint(low=0, high=dataset.n_points-1,
                            size=int(dataset.n_points * subset))
    return dataset.points[ids]


points = generate_points()

In [ ]:
point_cloud = pv.PolyData(points)
#point_cloud

In [ ]:
point_cloud.plot(eye_dome_lighting=True, notebook=False)

In [ ]:
mesh2
mesh3 = mesh2.compute_cell_sizes(length=False, volume=False)
sumesh3rf = mesh3.cell_data_to_point_data()

In [ ]:
#vol.plot(volume=True, cmap="bone", cpos=cpos)